## Examples used in Lecture 6

I only presented a few examples in class.

In [0]:
file_path = "/Volumes/workspace/default/files/train.csv"

df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)
df.columns

This is an example of a binary expression (i.e., predicate) that evaluates to true when Pclass is not 3.  filter returns a DataFrame.

We then group rows based on Pclass which creates two groups: one for first class, one for second class.  groupBy does NOT return a Dataframe.  Instead it returns a GroupedData object.  I found this a bit confusing because I expected filter to return a DataFrame.

In [0]:
from pyspark.sql.functions import col
print(type(df.filter(col("Pclass") != 3)))
print(type(df.filter(col("Pclass") != 3).groupBy("Pclass")))

Now I want to count the number of passengers in each group.  This should tell me how many passengers were in first class and how many in second class, but it didn't do exactly as I expected.

In [0]:
from pyspark.sql.functions import col, count, when
df.filter(col("Pclass") != 3).groupBy("Pclass").count()

The return result of `count` is not the number of rows with Pclass = 1 and the number of rows with Pclass. = 2.   Instead `count` returns a `DataFrame`.  I believed that `count` is an action and thus should actually perform the count, but it did not.   This is because the method of `GroupedData` is not an action... it is a transformation.   The `count` method of `DataFrame` is an action.   Using `count` as a transformation and an action in another is somewhat confusing.  To make it even more confusion there is also a function in `pyspark.sql.functions` named `count` which creates an unresolved expression.

#### DataFrame.count

```
df.count()
```

* Method on DataFrame
* Action: triggers execution
* Returns an integer

#### GroupedData.count

```
df.groupBy("Pclass").count()
```

* Method on GroupedData
* Returns a new DataFrame
* Transformation and thus lazy
* Adds an Aggregate node to the logical plan


#### functions.count

```
from pyspark.sql.functions import count

df.groupBy("Pclass").agg(count("Age"))
```

* `count("Age") is an unresolved expression representing the count of the number of rows with non-null Age.
* returns a `Column` object.
* becomes part of the logical plan
* `agg` return a DataFrame that is has not yet been evaluated.  It is still lazy until an action.


#### Now let's add an action.

When we add `show` it forces us to generate results.  The result is a count of the number of passengers in each group where each group is represents either `Pclass == 1` or `Pclass == 2`.

In [0]:
df.filter(col("Pclass") != 3).groupBy("Pclass").count().show()